In [4]:
import os
import numpy as np
import pandas as pd
os.chdir("/Users/jsusser/Desktop/code/funds/")


hf = pd.DataFrame({})
my_dir = os.listdir()
my_dir.remove(".DS_Store")
for x in my_dir:
    print(x)
    df = pd.read_csv(x)
    df.columns = df.iloc[0]
    df = df[1:]
    name = x.strip(".csv")
    cf = pd.DataFrame({name: df["ID.WEIGHTS"]})
    cf.index = df.iloc[:,-1].replace(np.nan, "N/A")
    cf = cf.dropna(axis=0)
    na = cf[cf.index=="N/A"].astype('float').sum(axis=0)
    cf = cf[cf.index != "N/A"]
    cf.loc["N/A"] = na
    hf = hf.merge(cf, how='outer', left_index=True, right_index=True)
    

hf = hf.fillna(0)

#hf.index = pd.Series(hf.index).replace(np.nan, "N/A")
#hf = hf.reindex(columns=["aod","adx","peo", "aio"])
hf = hf.astype('float')


sf = pd.read_csv("../s&p500.csv")
sf = sf[["Symbol","Weight"]]
#https://www.slickcharts.com/sp500

adx.csv
asa.csv
agd.csv
aio.csv
aod.csv
peo.csv


In [12]:

listt=[]
big_num = 2
def get_nums(num,pre_nums=[]):
    if num == 1:
        h = round(1 - sum(pre_nums),2)
        p = pre_nums + [h]
        listt.append(p)
        return p
    left = int(101 - (sum(pre_nums)*100))
    for x in range(0,left):
        pre_nums.append(x/100)

        get_nums(num-1,pre_nums)
        pre_nums = pre_nums[:0-num+1]
        
    return listt
listt=[]
big_num = 2

hey = get_nums(4)
len(hey)

176376

In [14]:
listt=[]
def get_nums(num,pre_nums=[]):
    if num == 1:
        h = round(1 - sum(pre_nums),1)
        p = pre_nums + [h]
        listt.append(p)
        return p
    left = int(11 - (sum(pre_nums)*10))
    for x in range(0,left):
        pre_nums.append(x/10)

        get_nums(num-1,pre_nums)
        pre_nums = pre_nums[:0-num+1]
        
    return listt
hey = get_nums(6)
len(hey)

2902

In [15]:
def find(weights, hf, sf):
    
    
    df = hf.multiply(weights).sum(axis=1).to_frame("ours")
    df = df.merge(sf, how="outer", right_on='Symbol', left_index=True)
    df = df.set_index("Symbol")
    #df = df.drop(["Symbol"], axis=1)
    df = df.fillna(0)
    return round(sum(abs(df["ours"]-df["Weight"])),5)

    

    
def start():
    k = 1000 
    for weights in hey:
        h = find(weights, hf, sf)
        if h<k:
            k=h
            l = weights
    print(k)
    print(l)
    
#print(find(weights, hf, sf))
weights = [1/len(hf.columns) for x in range(len(hf.columns))]
start()

#[0.1, 0.03, 0.82, 0.05]


89.34643
[0.8, 0.0, 0.0, 0.1, 0.0, 0.1]


In [16]:


def change_down(weights, find, no_change_nums):
    k = [x + (.01/(len(weights)-len(no_change_nums)-1)) for x in weights[:find]+weights[find+1:]]
    k.insert(find, weights[find]-.01)
    for x in no_change_nums:
        k[x] = weights[x]
    
    return k
def change_up(weights, find, no_change_nums):
    k = [x - (.01/(len(weights)-len(no_change_nums)-1)) for x in weights[:find]+weights[find+1:]]
    k.insert(find, weights[find]+.01)
    for x in no_change_nums:
        k[x] = weights[x]
    
    
    return k

def find(weights):
    global hf
    global sf
    df = hf.multiply(weights).sum(axis=1).to_frame("ours")
    df = df.merge(sf, how="outer", right_on='Symbol', left_index=True)
    df = df.set_index("Symbol")
    #df = df.drop(["Symbol"], axis=1)
    df = df.fillna(0)
    return sum(abs(df["ours"]-df["Weight"]))

def best(type, weights, ideal, ideal_weights):
    global no_change_nums
    global find_num
    num = find(weights)
    if num < ideal:
        if type == "up":
            if weights[find_num] + .01 > 0:
                return best("up",  change_up(weights, find_num, no_change_nums), num, weights)
            else:
                return weights
        if type == "down":
            if weights[find_num] - .01 > 0:
                return best("down", change_down(weights, find_num, no_change_nums), num, weights)
            else: 
                return weights
    else:
        return ideal_weights
    
    
    


def go_through(weights):
    global hf
    global sf
    global no_change_num
    global find_num
    num = find(weights)
    dw = change_down(weights, find_num, no_change_nums)
    uw = change_up(weights, find_num, no_change_nums)
    up = find(uw)
    down = find(dw)
    if up < num:
        return best("up", uw,num, [])
        
    elif down < num:
        return best("down", dw,num, [])
    else:
        return weights
    
        




In [77]:
weights = [1/len(hf.columns) for x in range(len(hf.columns))]
way = [[0, []], [1, [0]], [0,[1]], [0,[1]], [2,[0,1]], [1,[0,2]], [0,[1,2]], [2,[]], [1,[0,2]], [0,[1,2]]]

for y in range(2):
    for x in way:
        find_num = x[0]
        no_change_nums = x[1]
        weights = go_through(weights)
print(weights)

[0.029999999999999916, 0.8233333333333341, 0.0533333333333335, 0.09333333333333349]


In [74]:
[0.1, 0.03, 0.82, 0.05]

[0.1, 0.03, 0.82, 0.05]

In [22]:
t = []
z = 5

def path(y):
    global z
    if y == z:
        return "hey"
    global t
    for x in range(y,-1,-1):
        h = [x for x in range(y+1)]
        h.remove(x)
        t.append([x, h])
    path(y+1)
    if y == 0:
        return t
        
    
        

In [23]:
way = path(0)
way

[[0, []],
 [1, [0]],
 [0, [1]],
 [2, [0, 1]],
 [1, [0, 2]],
 [0, [1, 2]],
 [3, [0, 1, 2]],
 [2, [0, 1, 3]],
 [1, [0, 2, 3]],
 [0, [1, 2, 3]],
 [4, [0, 1, 2, 3]],
 [3, [0, 1, 2, 4]],
 [2, [0, 1, 3, 4]],
 [1, [0, 2, 3, 4]],
 [0, [1, 2, 3, 4]]]

In [44]:
weights = [1/len(hf.columns) for x in range(len(hf.columns))]
for y in range(10):
    for x in way:
        find_num = x[0]
        no_change_nums = x[1]
        weights = go_through(weights)
print(weights)

[0.8166666666666672, 0.0006666666666665343, 0.0031666666666665434, 0.10649999999999986, 0.016499999999999883, 0.05649999999999989]


In [ ]:
[0.8, 0.0, 0.0, 0.1, 0.0, 0.1]

In [43]:
way = [[0, []],
 [1, [0]],
 [0, [1]],
 [2, [0, 1]],
 [1, [0, 2]],
 [0, [1, 2]],
 [3, [0, 1, 2]],
 [2, [0, 1, 3]],
 [1, [0, 2, 3]],
 [0, [1, 2, 3]],
 [4, [0, 1, 2, 3]],
 [3, [0, 1, 2, 4]],
 [2, [0, 1, 3, 4]],
 [1, [0, 2, 3, 4]],
 [0, [1, 2, 3, 4]]]

In [46]:
weights = [round(x,3) for x in weights]
weights

[0.817, 0.001, 0.003, 0.106, 0.016, 0.056]